Convolutions:

Definition: Convolutions are mathematical operations applied to each pixel in an image using a convolution kernel or filter. It involves sliding a small matrix over the image and performing operations like weighted sums.

Purpose: Convolutions are used for various image processing tasks, including smoothing, sharpening, edge detection, and feature extraction.

Use Case: In satellite image processing, convolutions are often employed for enhancing specific features, reducing noise, and extracting valuable information from the imagery.

In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=JQsj0Jty5v3tvYIwwuJCjHIoj8a7Fuwwyn0a9WNDVyQ&tc=UhULv_kKOPb8NAF8fkn_gq0fqGCPDToWL9CVb79AwVY&cc=O5Wum8mZDiy7iKX_PZ-R54nQTU9OHHQww8T1SnlpCT4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkhtFIkygR16BTG6sY0U49bIYF32f_mUyI-2og3xMqVHnfmGQlBDEI

Successfully saved authorization token.


In [3]:
#Load Region Data
region = ee.Geometry.Point([2.3986, 47.0833])

In [4]:
#Landsat Data
rawImage = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")\
.filterDate('2016-01-01', '2016-12-31')\
.filterBounds(region)

In [5]:
# Median over time, in each band and pixel
median = rawImage.median()

In [6]:
#Visual Codes
visParams = {
    'bands': ['B4', 'B3', 'B2'],
    'max': 0.3
}

kernel = ee.Kernel.square(**{
    'radius': 5,
    'units': 'pixels',
    'normalize': True
})

In [7]:
#Convolution Code
smoothImage = median.convolve(kernel)

In [8]:
#Folium and codes
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [9]:
# Create a folium map object.
my_map = folium.Map(location=([47.0833, 2.3986]), zoom_start = 5)

# Add the layer to the map object.
my_map.add_ee_layer(median, visParams, 'Raw Image')
my_map.add_ee_layer(smoothImage, visParams, 'Smoothed Image')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)